Alunos:

- Bruno Santos (bfss@cesar.school)
- Caio Guedes (ccsg@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [2]:
!pip install langchain langchain_openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00


In [8]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.0 MB/s eta 0:00:00


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [4]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [6]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [9]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [10]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [11]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [13]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [16]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde e odontológico como parte de seu pacote de benefícios. Além disso, há outros auxílios como vale-refeição, auxílio home office e seguro de vida.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [34]:
import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import TextLoader

In [35]:
# Lista de URLs das páginas da Wiki
urls = [
    "https://pt.wikipedia.org/wiki/League_of_Legends"
]

# Nome do arquivo
output_file = "lol.txt"

# Abrir o arquivo no modo de escrita
with open(output_file, "w", encoding="utf-8") as file:
    for url in urls:
        try:
            # Fazer a requisição e extrair o conteúdo
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extrair o texto da página
            content = soup.get_text()

            # Escrever o título da página como um separador
            file.write(f"\n\n--- Conteúdo da página: {url} ---\n\n")
            file.write(content)

            print(f"Conteúdo da URL {url} salvo com sucesso!")
        except Exception as e:
            print(f"Erro ao processar a URL {url}: {e}")

print(f"Todo o conteúdo foi salvo em {output_file}.")

Conteúdo da URL https://pt.wikipedia.org/wiki/League_of_Legends salvo com sucesso!
Todo o conteúdo foi salvo em lol.txt.


In [36]:
# Carregar o arquivo .txt
loader = TextLoader("lol.txt")
docs = loader.load()

print(f"Documento carregado com {len(docs)} página.")

Documento carregado com 1 página.


Texto com muita informação porém bem prolixo.

In [40]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

225

In [41]:
print(all_splits[40].page_content)

League of Legends foi anunciado em 7 de outubro de 2008, para Microsoft Windows.[44][45] O teste beta fechado começou em abril de 2009.[44][46] Após o lançamento da beta, dezessete campeões estavam disponíveis.[47] A Riot inicialmente pretendia lançar o jogo com vinte campeões, mas dobrou o número antes de seu lançamento completo na América do Norte em 27 de outubro de 2009.[48][49] O título completo do jogo foi anunciado como League of Legends: Clash of Fates; a Riot planejava usar o subtítulo


In [42]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [50]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("quais modos de jogo do LoL?")

len(retrieved_docs)

6

In [51]:
print(retrieved_docs[0].page_content)

Outros modos
Além de Summoner's Rift, League of Legends possui dois outros modos de jogo permanentes. ARAM ("All Random, All Mid") é um modo cinco-contra-cinco como Summoner's Rift, mas em um mapa chamado Howling Abyss com apenas uma longa rota, sem área de selva e com campeões escolhidos aleatoriamente para os jogadores.[23][24][25] Dado o pequeno tamanho do mapa, os jogadores devem estar vigilantes para evitar as habilidades inimigas.[26]


In [52]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [53]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


In [54]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [55]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [57]:
for chunk in rag_chain.stream("Quais modos de jogo tem no LoL?"):
    print(chunk, end="", flush=True)

League of Legends tem três modos de jogo permanentes: Summoner's Rift, ARAM ("All Random, All Mid") e modos ranqueados. Além disso, há modos temporários que são frequentemente alinhados a eventos do jogo.